In [64]:
from collections import defaultdict as dd
import os
import json
import sqlite3
import ast
import re
from sqlite3 import Error
from collections import Counter

import sys
sys.path.append("rat-sql")
sys.path.append("rat-sql")
from ratsql.datasets.spider_lib.process_sql import get_schema, Schema, get_sql
from ratsql.datasets.spider_lib.evaluation import Evaluator

from ratsql import datasets
from ratsql.utils import registry
from ratsql.datasets.spider_lib import evaluation

In [36]:
config_temp = {
            "db_path": "rat-sql/data/spider/database",
            "name": "spider",
            "paths": [
                "rat-sql/data/spider/train_spider.json",
                "rat-sql/data/spider/train_others.json"
            ],
            "tables_paths": [
                "rat-sql/data/spider/tables.json"
            ]
        }

data = registry.construct('dataset', config_temp)

Exception ignored in: <function SpiderDataset.__del__ at 0x7f28d08e1ef0>
Traceback (most recent call last):
  File "/repo/rat-sql/ratsql/datasets/spider.py", line 167, in __del__
    for _, schema in self.schemas.items():
AttributeError: 'SpiderDataset' object has no attribute 'schemas'
DB connections: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 166/166 [00:00<00:00, 264.03it/s]


In [152]:
spider_db_loc = 'rat-sql/data/spider/database/'

ratsql_infereval_loc = 'rat-sql/logdir/bert_run/bs=12,lr=7.4e-04,bert_lr=3.0e-06,end_lr=0e0,att=1/ie_dirs/'
ratsql_train = 'rat-sql/data/spider/train_spider.json'
ratsql_train_others = 'rat-sql/data/spider/train_others.json'
ratsql_infer = os.path.join(ratsql_infereval_loc, 'bert_run_true_1-step30100.infer')
ratsql_eval  = os.path.join(ratsql_infereval_loc, 'bert_run_true_1-step30100.eval')

In [49]:
db_path = os.path.join(spider_db_loc, 'department_management/department_management.sqlite')
schema = Schema(get_schema(db_path))
gold = 'SELECT count(*) FROM head WHERE age  >  56'

g_sql = get_sql(schema, gold)

foreign_key_maps = {
    db_id: evaluation.build_foreign_key_map(schema.orig)
    for db_id, schema in data.schemas.items()
}
evaluator = evaluation.Evaluator(
    data.db_path,
    foreign_key_maps,
    'match')

evaluator.eval_hardness(g_sql) 


In [50]:
train_dataset = read_json(ratsql_train)
train_others = read_json(ratsql_train_others)

train_all = train_dataset + train_others
print(len(train_all))

8659


In [61]:
train_hardness = []
failed = 0

for data in train_all:
    try:
        db_id = data['db_id']
        db_path = os.path.join(spider_db_loc, f'{db_id}/{db_id}.sqlite')
        schema = Schema(get_schema(db_path))
        gold = data['query']

        g_sql = get_sql(schema, gold)

        train_hardness.append(evaluator.eval_hardness(g_sql))
    except Exception as e:
        print(e)
        failed += 1

train_hardness_dic = Counter(train_hardness)

'ref_company_types'


In [3]:
db_length = len(os.listdir('rat-sql/data/spider/database/'))
print(f'How many DB: {db_length}')

How many DB: 166


In [117]:
def read_json(file):
    with open(file) as json_file:
        json_data = json.load(json_file)
    
    return json_data

def read_jsonl(file):
    with open(file) as json_file:
        json_data = [json.loads(line) for line in json_file]
    
    return json_data

def desc_by_hardness(total_scores, key):
    return total_scores[key]

def get_table_desc(db):
    schema = {}
    conn = sqlite3.connect(db)
    conn.text_factory = str
    cursor = conn.cursor()

    # fetch table names
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = [str(table[0].lower()) for table in cursor.fetchall()]

    for table in tables:
        schema[table] = {'columns': [], 'col_length': 0, 'row_length': 0}
        cursor.execute("PRAGMA table_info({})".format(table))
        schema[table]['columns'] = [str(col[1].lower()) for col in cursor.fetchall()]

        cursor.execute(f"select * from {table}")

        schema[table]['col_length'] = len(schema[table]['columns'])
        schema[table]['row_length'] = len(cursor.fetchall())

    return schema

def gold_table_cnt(item):
    gold = item['gold']

    p = re.compile('T[0-9]')
    used_table = set(list(p.findall(gold)))

    print(gold)
    print(used_table)
    print(len(used_table) if used_table else 0)
    
    return used_table

In [5]:
infer = read_jsonl(ratsql_infer)
eval  = read_jsonl(ratsql_eval)[0]

eval_peritem = eval['per_item']
eval_total_scores = eval['total_scores']

In [6]:
print(f"Easy Accuracy: {desc_by_hardness(eval_total_scores, 'easy')['exact']}")
print(f"Medium Accuracy: {desc_by_hardness(eval_total_scores, 'medium')['exact']}")
print(f"Hard Accuracy: {desc_by_hardness(eval_total_scores, 'hard')['exact']}")
print(f"Extra Accuracy: {desc_by_hardness(eval_total_scores, 'extra')['exact']}")

Easy Accuracy: 0.7741935483870968
Medium Accuracy: 0.547085201793722
Hard Accuracy: 0.5114942528735632
Extra Accuracy: 0.3433734939759036


In [73]:
dev_hardness_dic = {'easy':0, 'medium':0, 'hard':0, 'extra':0}

for i in ['easy', 'medium', 'hard', 'extra']:
    cnt = desc_by_hardness(eval_total_scores, i)['count']
    dev_hardness_dic[i] += cnt

In [76]:
train_hardness_dic = Counter(train_hardness)

for key in dev_hardness_dic:
    train_hardness_dic[key] += dev_hardness_dic[key]

train_hardness_dic

Counter({'easy': 2231, 'medium': 3445, 'hard': 2095, 'extra': 1921})

In [ ]:
infer = read_jsonl(ratsql_infer)
eval  = read_jsonl(ratsql_eval)[0]

eval_peritem = eval['per_item']
eval_total_scores = eval['total_scores']

In [78]:
exact_true = []
exact_false = []

for i in eval_peritem:
    if i['exact']:
        exact_true.append(i)
    else:
        exact_false.append(i)

In [167]:
from collections import defaultdict as dd


dict = dd(lambda: {})

In [163]:
from collections import defaultdict as dd

level = {'easy': 0, 'medium': 0, 'hard': 0, 'extra': 0}
cnt = {'easy':0, 'medium': 0, 'hard': 0, 'extra': 0}
for i in eval_peritem:
    rule = re.compile('T[1-9]')
    found = len(set(rule.findall(i['gold'])))
    if found:
        level[i['hardness']] += found
    else:
        level[i['hardness']] += 1
    cnt[i['hardness']] += 1


In [172]:
print(f'Level cnt: {level}')
print(f'Table Count: {cnt}')
for key in cnt:
    print(f'Table Mean: {level[key] / cnt[key]}')

Level cnt: {'easy': 254, 'medium': 629, 'hard': 263, 'extra': 335}
Table Count: {'easy': 248, 'medium': 446, 'hard': 174, 'extra': 166}
Table Mean: 1.0241935483870968
Table Mean: 1.4103139013452914
Table Mean: 1.5114942528735633
Table Mean: 2.0180722891566263


In [81]:
correct_easy, correct_medium, correct_hard, correct_extra = 0,0,0,0

for i in exact_true:
    if i['hardness'] == 'easy':
        correct_easy += 1
    elif i['hardness'] == 'medium':
        correct_medium += 1
    elif i['hardness'] == 'hard':
        correct_hard += 1
    else:
        correct_extra += 1

print(correct_easy, correct_medium, correct_hard, correct_extra)

192 244 89 57


In [83]:
wrong_easy, wrong_medium, wrong_hard, wrong_extra = 0,0,0,0

for i in exact_false:
    if i['hardness'] == 'easy':
        wrong_easy += 1
    elif i['hardness'] == 'medium':
        wrong_medium += 1
    elif i['hardness'] == 'hard':
        wrong_hard += 1
    else:
        wrong_extra += 1

print(wrong_easy, wrong_medium, wrong_hard, wrong_extra)

56 202 85 109


In [121]:
wrong_easy, wrong_medium, wrong_hard, wrong_extra = [],[],[],[]
error = 0

for wrong in exact_false:
    db_name = wrong['db_name']
    db_path = os.path.join(spider_db_loc, f'{db_name}/{db_name}.sqlite')
    hardness = wrong['hardness']

    try:
        desc = get_table_desc(db_path)

        if hardness == 'easy' and desc not in wrong_easy:
            wrong_easy.append(desc)
        elif hardness == 'medium' and desc not in wrong_medium:
            wrong_medium.append(desc)
        elif hardness == 'hard' and desc not in wrong_hard:
            wrong_hard.append(desc)
        elif hardness == 'extra' and desc not in wrong_extra:
            wrong_extra.append(desc)
    except Exception as e:
        # print(e)
        error += 1

In [271]:
db_name = 'cre_Doc_Template_Mgt'
db_path = os.path.join(spider_db_loc, f'{db_name}/{db_name}.sqlite')

get_table_desc(db_path)

{'ref_template_types': {'columns': ['template_type_code',
   'template_type_description'],
  'col_length': 2,
  'row_length': 5},
 'templates': {'columns': ['template_id',
   'version_number',
   'template_type_code',
   'date_effective_from',
   'date_effective_to',
   'template_details'],
  'col_length': 6,
  'row_length': 20},
 'documents': {'columns': ['document_id',
   'template_id',
   'document_name',
   'document_description',
   'other_details'],
  'col_length': 5,
  'row_length': 15},
 'paragraphs': {'columns': ['paragraph_id',
   'document_id',
   'paragraph_text',
   'other_details'],
  'col_length': 4,
  'row_length': 15}}

In [174]:
# 난이도 별로 어떤 Clause에서 오류를 범하는가?
wrong_easy, wrong_medium, wrong_hard, wrong_extra = [],[],[],[]
error = 0

for wrong in exact_false:
    db_name = wrong['db_name']
    db_path = os.path.join(spider_db_loc, f'{db_name}/{db_name}.sqlite')
    hardness = wrong['hardness']

    try:
        desc = get_table_desc(db_path)

        if hardness == 'easy':
            wrong_easy.append(desc)
        elif hardness == 'medium':
            wrong_medium.append(desc)
        elif hardness == 'hard':
            wrong_hard.append(desc)
        elif hardness == 'extra':
            wrong_extra.append(desc)
    except Exception as e:
        # print(e)
        error += 1

[{'stadium': {'columns': ['stadium_id',
    'location',
    'name',
    'capacity',
    'highest',
    'lowest',
    'average'],
   'col_length': 7,
   'row_length': 9},
  'singer': {'columns': ['singer_id',
    'name',
    'country',
    'song_name',
    'song_release_year',
    'age',
    'is_male'],
   'col_length': 7,
   'row_length': 6},
  'concert': {'columns': ['concert_id',
    'concert_name',
    'theme',
    'stadium_id',
    'year'],
   'col_length': 5,
   'row_length': 6},
  'singer_in_concert': {'columns': ['concert_id', 'singer_id'],
   'col_length': 2,
   'row_length': 10}},
 {'student': {'columns': ['stuid',
    'lname',
    'fname',
    'age',
    'sex',
    'major',
    'advisor',
    'city_code'],
   'col_length': 8,
   'row_length': 34},
  'has_pet': {'columns': ['stuid', 'petid'], 'col_length': 2, 'row_length': 3},
  'pets': {'columns': ['petid', 'pettype', 'pet_age', 'weight'],
   'col_length': 4,
   'row_length': 3}},
 {'continents': {'columns': ['contid', 'conti

In [127]:
partial_keys = exact_false[0]['partial'].keys()

In [176]:
partial_dic = {'easy':{}, 'medium': {}, 'hard':{}, 'extra':{}}
for kk in partial_dic:
    for k in partial_keys:
        partial_dic[kk][k] = 0

In [178]:
partial_dict = {'easy':{}, 'medium': {}, 'hard':{}, 'extra':{}}
for kk in partial_dict:
    for k in partial_keys:
        partial_dict[kk][k] = 0

In [179]:
for data in exact_false:
    hardness = data['hardness']
    
    for part_key in data['partial']:
        print(data['partial'][part_key]['acc'])
        if data['partial'][part_key]['acc']:
            partial_dic[hardness][part_key] += 1
        elif not data['partial'][part_key]['acc']:
            partial_dict[hardness][part_key] += 1

1
1
1
1
1
1
0
1
1
0
0
0
1
1
1
1
1
1
1
1
0
0
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
0
0
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
1
1
0
1
1
1
1
0
0
0
1
1
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
1
1
0
1
1
0
0
1
1
1
1
0
0
0
0
1
1
1
1
1
1
1
1
1
1
0
0
1
1
1
1
1
1
1
1
0
0
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
0
0
1
1
1
1
1
1
1
1
0
0
1
1
1
1
1
1
1
1
1
1
0
0
1
1
1
1
1
1
1
1
0
0
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
0
0
1
1
1
1
0
0
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
0
1
1
1
0
0
1
1
1
0
0
0
0
0
0
0
1
1
1
0
0
0
0
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
1
1
1
1
1
1
1
1
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
1
1
1
0
1
1
1
0
1
1
0
0
1
1
1
0
1
1
1
1
0
0
1
1
1
0
1
1
0
0
1
1
1
1
1
1
1
1
0
0
1
1
1
1
1
1
1
1
0
0
1
1
1
1
1
1
1
1
0
0
1
1
1
1
1
1
1
1
0
0
1
1
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
1
1
0
1
1
0
0
0
0
0
1
1
0
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
1
1
0
0
1
1
1
1
0
0
1
1
0
0
1
1
1
1
1
1
0
0
1
1
1
0
1
1
1
1
0
0
1
1
1
0
1
1


In [180]:
partial_dict

{'easy': {'select': 23,
  'select(no AGG)': 15,
  'where': 15,
  'where(no OP)': 13,
  'group(no Having)': 4,
  'group': 7,
  'order': 6,
  'and/or': 4,
  'IUEN': 4,
  'keywords': 14},
 'medium': {'select': 94,
  'select(no AGG)': 82,
  'where': 48,
  'where(no OP)': 44,
  'group(no Having)': 45,
  'group': 50,
  'order': 37,
  'and/or': 20,
  'IUEN': 4,
  'keywords': 51},
 'hard': {'select': 4,
  'select(no AGG)': 4,
  'where': 50,
  'where(no OP)': 41,
  'group(no Having)': 10,
  'group': 10,
  'order': 9,
  'and/or': 10,
  'IUEN': 38,
  'keywords': 46},
 'extra': {'select': 32,
  'select(no AGG)': 27,
  'where': 49,
  'where(no OP)': 41,
  'group(no Having)': 24,
  'group': 29,
  'order': 23,
  'and/or': 20,
  'IUEN': 29,
  'keywords': 46}}

In [ ]:
db_path = os.path.join(spider_db_loc, '')

In [149]:
train_dataset =read_json('rat-sql/data/spider/tables.json')

In [150]:
train_dataset[0].keys()


dict_keys(['column_names', 'column_names_original', 'column_types', 'db_id', 'foreign_keys', 'primary_keys', 'table_names', 'table_names_original'])

In [151]:
train_dataset[0]

{'column_names': [[-1, '*'],
  [0, 'perpetrator id'],
  [0, 'people id'],
  [0, 'date'],
  [0, 'year'],
  [0, 'location'],
  [0, 'country'],
  [0, 'killed'],
  [0, 'injured'],
  [1, 'people id'],
  [1, 'name'],
  [1, 'height'],
  [1, 'weight'],
  [1, 'home town']],
 'column_names_original': [[-1, '*'],
  [0, 'Perpetrator_ID'],
  [0, 'People_ID'],
  [0, 'Date'],
  [0, 'Year'],
  [0, 'Location'],
  [0, 'Country'],
  [0, 'Killed'],
  [0, 'Injured'],
  [1, 'People_ID'],
  [1, 'Name'],
  [1, 'Height'],
  [1, 'Weight'],
  [1, 'Home Town']],
 'column_types': ['text',
  'number',
  'number',
  'text',
  'number',
  'text',
  'text',
  'number',
  'number',
  'number',
  'text',
  'number',
  'number',
  'text'],
 'db_id': 'perpetrator',
 'foreign_keys': [[2, 9]],
 'primary_keys': [1, 9],
 'table_names': ['perpetrator', 'people'],
 'table_names_original': ['perpetrator', 'people']}

Table 개수가 성능에 미치는 영향

In [185]:
get_table_desc(os.path.join(spider_db_loc, 'flight_1/flight_1.sqlite'))

{'flight': {'columns': ['flno',
   'origin',
   'destination',
   'distance',
   'departure_date',
   'arrival_date',
   'price',
   'aid'],
  'col_length': 8,
  'row_length': 10},
 'aircraft': {'columns': ['aid', 'name', 'distance'],
  'col_length': 3,
  'row_length': 16},
 'employee': {'columns': ['eid', 'name', 'salary'],
  'col_length': 3,
  'row_length': 31},
 'certificate': {'columns': ['eid', 'aid'], 'col_length': 2, 'row_length': 69}}

난이도 별 Question / SQL Length

In [192]:
eval_peritem[20]

{'predicted': "SELECT Count(*) FROM concert WHERE concert.Year = 'terminal' OR concert.Year = 'terminal'",
 'gold': 'SELECT count(*) FROM concert WHERE YEAR  =  2014 OR YEAR  =  2015',
 'predicted_parse_error': False,
 'hardness': 'medium',
 'exact': True,
 'partial': {'select': {'acc': 1,
   'rec': 1,
   'f1': 1,
   'label_total': 1,
   'pred_total': 1},
  'select(no AGG)': {'acc': 1,
   'rec': 1,
   'f1': 1,
   'label_total': 1,
   'pred_total': 1},
  'where': {'acc': 1, 'rec': 1, 'f1': 1, 'label_total': 2, 'pred_total': 2},
  'where(no OP)': {'acc': 1,
   'rec': 1,
   'f1': 1,
   'label_total': 2,
   'pred_total': 2},
  'group(no Having)': {'acc': 1,
   'rec': 1,
   'f1': 1,
   'label_total': 0,
   'pred_total': 0},
  'group': {'acc': 1, 'rec': 1, 'f1': 1, 'label_total': 0, 'pred_total': 0},
  'order': {'acc': 1, 'rec': 1, 'f1': 1, 'label_total': 0, 'pred_total': 0},
  'and/or': {'acc': 1, 'rec': 1, 'f1': 1, 'label_total': 1, 'pred_total': 1},
  'IUEN': {'acc': 1, 'rec': 1, 'f1': 1,

In [194]:
infer_data = read_jsonl(ratsql_infer)

In [265]:
def length(items):
    level = {'easy': 0, 'medium': 0, 'hard': 0, 'extra': 0}
    cnt = {'easy':0, 'medium': 0, 'hard': 0, 'extra': 0}  
    mean = {'easy':0, 'medium': 0, 'hard': 0, 'extra': 0}  

    s_level = {'easy': 0, 'medium': 0, 'hard': 0, 'extra': 0}
    s_cnt = {'easy':0, 'medium': 0, 'hard': 0, 'extra': 0}  
    s_mean = {'easy':0, 'medium': 0, 'hard': 0, 'extra': 0}  

    for data in items:
        try:
            question_length = len(data['question_toks'])
            sql = data['query_toks']
            rule = re.compile('T[1-9]')
            sub_sql = [re.sub(rule, '', s) for s in sql]
            
            sql_length = len([i for i in sql if i.isupper()])

            db_id = data['db_id']
            db_path = os.path.join(spider_db_loc, f'{db_id}/{db_id}.sqlite')
            schema = Schema(get_schema(db_path))
            gold = data['query']

            g_sql = get_sql(schema, gold)

            hardness = evaluator.eval_hardness(g_sql)

            level[hardness] += question_length
            cnt[hardness] += 1  

            mean[hardness] = level[hardness] / cnt[hardness]

            s_level[hardness] += sql_length
            s_cnt[hardness] += 1  

            s_mean[hardness] = s_level[hardness] / s_cnt[hardness]
 
        except Exception as e:
            print(e)
            pass

    return level, cnt, mean, s_level, s_cnt, s_mean


In [266]:
level, cnt, mean, s_level, s_cnt, s_mean = length(train_all)

'ref_company_types'


In [267]:
s_mean

{'easy': 3.626828038325769,
 'medium': 6.960320106702234,
 'hard': 9.975013014055179,
 'extra': 15.074074074074074}